In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud

# Importing nltk libraries
import re
import string
import missingno
import pandas_profiling
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import FreqDist

from scipy import stats
from scipy.stats import zscore
from scipy.sparse import hstack
import scikitplot as skplt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, precision_score, confusion_matrix, accuracy_score, classification_report

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier

import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")
import joblib

lemmatizer = nltk.stem.WordNetLemmatizer()

In [2]:
df = pd.read_csv(r"C:\Users\Rutuja\Downloads\Review_Rating_Datafile.csv")
df # checking the first five and last five rows of our dataset

,Unnamed: 0,Review_title,Review_text,Ratings
0,0,Just started system. It's getting over heated...,\n Just started system. It's getting over he...,2.0 out of 5 stars
1,1,"Fast, Lightweight, Great Screen.. But very fli...",\n - Build quality is bad. Cheapest quality p...,2.0 out of 5 stars
2,2,Warranty only for 9 months,\n While my experience with the laptop has be...,2.0 out of 5 stars
3,3,Diapointed with the laptop.,\n The camera doesn't work.. the keyboard is ...,2.0 out of 5 stars
4,4,Came without manual and dirty screen,\n Received this laptop without manual and in...,2.0 out of 5 stars
...,...,...,...,...
41397,41397,Very poor,Came with 16a plug and screen started flickeri...,1
41398,41398,Good,Moniter sounde not working. Speaker comblaind....,3
41399,41399,Pretty good,nice monitor 😊,4
41400,41400,Really Nice,good but sound qwality is just ok,4


In [3]:
df.drop(columns = 'Unnamed: 0', inplace = True)
print("We have {} Rows and {} Columns in our dataframe".format(df.shape[0], df.shape[1]))
df.head(20)

We have 41402 Rows and 3 Columns in our dataframe


,Review_title,Review_text,Ratings
0,Just started system. It's getting over heated...,\n Just started system. It's getting over he...,2.0 out of 5 stars
1,"Fast, Lightweight, Great Screen.. But very fli...",\n - Build quality is bad. Cheapest quality p...,2.0 out of 5 stars
2,Warranty only for 9 months,\n While my experience with the laptop has be...,2.0 out of 5 stars
3,Diapointed with the laptop.,\n The camera doesn't work.. the keyboard is ...,2.0 out of 5 stars
4,Came without manual and dirty screen,\n Received this laptop without manual and in...,2.0 out of 5 stars
5,cheap quality at high price loose fittings,\n &nbsp;Features are good but quality is ver...,2.0 out of 5 stars
6,Superb laptop but issue with this warranty ☹️☹️,"\n very fast and smooth laptop, but i purchas...",2.0 out of 5 stars
7,over heating,\n over heating and fan noise screen size is ...,2.0 out of 5 stars
8,My laptop warranty 2month back start why?,\n &nbsp;Amazon sell not good product<br>Why ...,2.0 out of 5 stars
9,I Didn't prefer to any one,\n Display height is too small. As like smart...,2.0 out of 5 stars


In [4]:
df.isna().sum() # checking for missing values

Review_title    2625
Review_text     2338
Ratings         2626
dtype: int64

In [5]:
missingno.matrix(df, figsize = (15,8), color=(0.25, 0.75, 0.25), fontsize=18)

<AxesSubplot:>

In [6]:
print("We have {} Rows and {} Columns in our dataframe before removing NaN".format(df.shape[0], df.shape[1]))
df.dropna(inplace=True)
print("We have {} Rows and {} Columns in our dataframe after removing NaN".format(df.shape[0], df.shape[1]))

We have 41402 Rows and 3 Columns in our dataframe before removing NaN
We have 38682 Rows and 3 Columns in our dataframe after removing NaN


In [7]:
df.isnull().sum()

Review_title    0
Review_text     0
Ratings         0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38682 entries, 0 to 41401
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Review_title  38682 non-null  object
 1   Review_text   38682 non-null  object
 2   Ratings       38682 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [9]:
df['Ratings'].unique()

array(['2.0 out of 5 stars', '3.0 out of 5 stars', '1.0 out of 5 stars',
       '5.0 out of 5 stars', '4.0 out of 5 stars', '5', '4', '2', '3',
       '1'], dtype=object)

In [10]:
df['Ratings'] = df['Ratings'].replace('1.0 out of 5 stars',1)
df['Ratings'] = df['Ratings'].replace('2.0 out of 5 stars',2)
df['Ratings'] = df['Ratings'].replace('3.0 out of 5 stars',3)
df['Ratings'] = df['Ratings'].replace('4.0 out of 5 stars',4)
df['Ratings'] = df['Ratings'].replace('5.0 out of 5 stars',5)
df['Ratings'] = df['Ratings'].astype('int')
df['Ratings'].unique()

array([2, 3, 1, 5, 4])

In [11]:
# Now combining the "Review_title" and "Review_text" columns into one single column called "Review"
df['Review'] = df['Review_title'].map(str)+' '+df['Review_text']
df

,Review_title,Review_text,Ratings,Review
0,Just started system. It's getting over heated...,\n Just started system. It's getting over he...,2,Just started system. It's getting over heated...
1,"Fast, Lightweight, Great Screen.. But very fli...",\n - Build quality is bad. Cheapest quality p...,2,"Fast, Lightweight, Great Screen.. But very fli..."
2,Warranty only for 9 months,\n While my experience with the laptop has be...,2,Warranty only for 9 months \n While my experi...
3,Diapointed with the laptop.,\n The camera doesn't work.. the keyboard is ...,2,Diapointed with the laptop. \n The camera doe...
4,Came without manual and dirty screen,\n Received this laptop without manual and in...,2,Came without manual and dirty screen \n Recei...
...,...,...,...,...
41397,Very poor,Came with 16a plug and screen started flickeri...,1,Very poor Came with 16a plug and screen starte...
41398,Good,Moniter sounde not working. Speaker comblaind....,3,Good Moniter sounde not working. Speaker combl...
41399,Pretty good,nice monitor 😊,4,Pretty good nice monitor 😊
41400,Really Nice,good but sound qwality is just ok,4,Really Nice good but sound qwality is just ok


In [12]:
df['Review'][0]

"Just started system.  It's getting over heated. Fan is making lot of noise. \n  Just started system.  It's getting over heated. Fan is making lot of noise.<br>Received product today 21.06.2021 and when checked warranty information, it says started on 31st May 2021\n"

In [13]:
df['Review'][1]

"Fast, Lightweight, Great Screen.. But very flimsy build \n  - Build quality is bad. Cheapest quality plastic used, laptop flexes, makes creaking sound every where.<br>- Trackpad is weird, feels like broken but works well<br>- Glossy screen but I knew this before buying<br><br>+ Fast, boots to desktop in 6 seconds (remove mcafee, bloatware), great battery life, beautiful FHD screen, light weight, nice speakers. Other than build, everything else feels great.<br><br>If build quality is a major concern stay away. Try to check in person before buying because Amazon doesn't allow return only replacement.\n"

In [14]:
df['Review'][2]

'Warranty only for 9 months \n  While my experience with the laptop has been good and no issues noticed in terms of performance or battery at this price.<br><br>The only thing i regret is buying it online on Amazon as the warranty received is only for 9 months. When you check the warranty on the date of delivery which was just next day of placing the order, the warranty shows as 9 months.<br><br>My advice is to avoid buying it on Amazon.\n'

In [15]:
'''Here I am defining a function to replace some of the contracted words to their full form and removing urls and some 
unwanted text'''

def decontracted(text):
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don’t", "do not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"im ", "i am", text)
    text = re.sub(r"yo ", "you ",text)
    text = re.sub(r"doesn’t", "does not",text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"<br>", " ", text)
    text = re.sub(r'http\S+', '', text) #removing urls
    return text

# Lowercasing the alphabets
df['Review'] = df['Review'].apply(lambda x : x.lower())
df['Review'] = df['Review'].apply(lambda x : decontracted(x))

# Removing punctuations from the review
df['Review'] = df['Review'].str.replace('[^\w\s]','')
df['Review'] = df['Review'].str.replace('\n',' ')

In [16]:
# Removing all the stopwords
stop = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [17]:
df['Review'][0]

'started system getting heated fan making lot noise started system getting heated fan making lot noise received product today 21062021 checked warranty information says started 31st may 2021'

In [18]:
df['Review'][1]

'fast lightweight great screen flimsy build build quality bad cheapest quality plastic used laptop flexes makes creaking sound every trackpad weird feels like broken works well glossy screen knew buying fast boots desktop 6 seconds remove mcafee bloatware great battery life beautiful fhd screen light weight nice speakers build everything else feels great build quality major concern stay away try check person buying amazon allow return replacement'

In [19]:
df['Review'][2]

'warranty 9 months experience laptop good issues noticed terms performance battery price thing regret buying online amazon warranty received 9 months check warranty date delivery next day placing order warranty shows 9 months advice avoid buying amazon'

In [20]:
# Defining functiom to convert nltk tag to wordnet tags
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
# Defining function to lemmatize our text
def lemmatize_sentence(sentence):
    # tokenize the sentence and find the pos_tag
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    # tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x : (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatize_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatize_sentence.append(word)
        else:
            lemmatize_sentence.append(lemmatizer.lemmatize(word,tag))
    return " ".join(lemmatize_sentence)    

df['Review'] = df['Review'].apply(lambda x : lemmatize_sentence(x))

In [21]:
df['Review'][0]

'start system get heat fan make lot noise start system get heat fan make lot noise receive product today 21062021 check warranty information say start 31st may 2021'

In [22]:
df['Review'][1]

'fast lightweight great screen flimsy build build quality bad cheap quality plastic use laptop flex make creak sound every trackpad weird feel like broken work well glossy screen know buy fast boots desktop 6 second remove mcafee bloatware great battery life beautiful fhd screen light weight nice speaker build everything else feel great build quality major concern stay away try check person buy amazon allow return replacement'

In [23]:
df['Review'][2]

'warranty 9 month experience laptop good issue notice term performance battery price thing regret buying online amazon warranty receive 9 month check warranty date delivery next day place order warranty show 9 month advice avoid buying amazon'

In [24]:
# Noise removal function
def scrub_words(text):
    # remove HTML markup
    text = re.sub("(<.*?>)", "", text)
    # remove non-ascii and digits
    text = re.sub("(\\W)", " ", text)
    text = re.sub("(\\d)", "", text)
    # remove white space
    text = text.strip()
    return text

df['Review'] = df['Review'].apply(lambda x : scrub_words(x))

In [25]:
df['Review'][0]

'start system get heat fan make lot noise start system get heat fan make lot noise receive product today  check warranty information say start st may'

In [26]:
df['Review'][1]

'fast lightweight great screen flimsy build build quality bad cheap quality plastic use laptop flex make creak sound every trackpad weird feel like broken work well glossy screen know buy fast boots desktop  second remove mcafee bloatware great battery life beautiful fhd screen light weight nice speaker build everything else feel great build quality major concern stay away try check person buy amazon allow return replacement'

In [27]:
df['Review'][2]

'warranty  month experience laptop good issue notice term performance battery price thing regret buying online amazon warranty receive  month check warranty date delivery next day place order warranty show  month advice avoid buying amazon'

In [28]:
# Creating column for word counts in the review text
df['Review_WC'] = df['Review'].apply(lambda x: len(str(x).split(' ')))
df[['Review_WC', 'Review']].head(10)

,Review_WC,Review
0,27,start system get heat fan make lot noise start...
1,67,fast lightweight great screen flimsy build bui...
2,37,warranty month experience laptop good issue n...
3,26,diapointed laptop camera work keyboard low qua...
4,25,come without manual dirty screen receive lapto...
5,33,cheap quality high price loose fitting nbspfea...
6,14,superb laptop issue warranty fast smooth lapto...
7,12,heat heat fan noise screen size small suitable...
8,14,laptop warranty month back start nbspamazon se...
9,7,prefer one display height small like smartphone


In [29]:
# Density plot and histogram of all word count
plt.figure(figsize=(10,7))
sns.distplot(df['Review_WC'], hist = True, kde = True,
            bins = int(180/5), color = 'darkblue',
            hist_kws = {'edgecolor':'black'},
            kde_kws = {'linewidth':4})
plt.show()

In [30]:
# Creating column for character counts in the review text
df['Review_CC'] = df['Review'].str.len()
df[['Review_CC','Review']].head(10)

,Review_CC,Review
0,148,start system get heat fan make lot noise start...
1,427,fast lightweight great screen flimsy build bui...
2,238,warranty month experience laptop good issue n...
3,166,diapointed laptop camera work keyboard low qua...
4,165,come without manual dirty screen receive lapto...
5,197,cheap quality high price loose fitting nbspfea...
6,89,superb laptop issue warranty fast smooth lapto...
7,70,heat heat fan noise screen size small suitable...
8,90,laptop warranty month back start nbspamazon se...
9,47,prefer one display height small like smartphone


In [31]:
# Density plot and histogram of all character count
plt.figure(figsize=(10,7))
sns.distplot(df['Review_CC'], hist = True, kde = True,
            bins = int(180/5), color = 'darkblue',
            hist_kws = {'edgecolor':'black'},
            kde_kws = {'linewidth':4})
plt.show()

In [32]:
# Applying zscore to remove outliers
z_score = zscore(df[['Review_WC']])
abs_z_score = np.abs(z_score)
filtering_entry = (abs_z_score < 3).all(axis = 1)
df = df[filtering_entry]
print("We have {} Rows and {} Columns in our dataframe after removing outliers".format(df.shape[0], df.shape[1]))

We have 38175 Rows and 6 Columns in our dataframe after removing outliers


In [33]:
pandas_profiling.ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
# Density plot and histogram of all word count
plt.figure(figsize=(10,7))
sns.distplot(df['Review_WC'], hist = True, kde = True,
            bins = int(180/5), color = 'darkblue',
            hist_kws = {'edgecolor':'black'},
            kde_kws = {'linewidth':4})
plt.show()

In [35]:
# Density plot and histogram of all character count
plt.figure(figsize=(10,7))
sns.distplot(df['Review_CC'], hist = True, kde = True,
            bins = int(180/5), color = 'darkblue',
            hist_kws = {'edgecolor':'black'},
            kde_kws = {'linewidth':4})
plt.show()

In [36]:
# Checking the ratings column details using count plot
x = 'Ratings'
fig, ax = plt.subplots(1,1,figsize=(10,8))
sns.countplot(x=x,data=df,ax=ax)
p=0
for i in ax.patches:
    q = i.get_height()/2
    val = i.get_height()
    ratio = round(val*100/len(df),2)
    prn = f"{val}\n({ratio}%)"
    ax.text(p,q,prn,ha="center",color="white",rotation=0,fontweight="bold",fontsize="13")
    p += 1
    
plt.title("Count Plot: rating")
plt.show()

In [37]:
# Checking review word count distribution for each rating
ratings = np.sort(df.Ratings.unique())
cols = 3
rows = len(ratings)//cols
if rows % cols != 0:
    rows += 1
    
fig = plt.figure(figsize=(20,10))
plt.subplots_adjust(hspace=0.3, wspace=0.2)
p = 1
colors = [(1,0,0,1),(0.6,0.2,0,1),(0.4,0.5,0,1),(0.2,0.7,0,1),(0,1,0.1,1)]
for i in ratings:
    axis = fig.add_subplot(rows,cols,p)
    sns.distplot(df.Review_WC[df.Ratings==i], ax=axis, label=f"For Rating: {i}", color=colors[i-1])
    axis.set_xlabel(f"Review Word Count")
    axis.legend()
    p += 1
    
plt.show()

In [38]:
# Checking review character count distribution for each rating
ratings = np.sort(df.Ratings.unique())
cols = 3
rows = len(ratings)//cols
if rows % cols != 0:
    rows += 1
    
fig = plt.figure(figsize=(20,10))
plt.subplots_adjust(hspace=0.3, wspace=0.2)
p = 1
colors = [(1,0,0,1),(0.6,0.2,0,1),(0.4,0.5,0,1),(0.2,0.7,0,1),(0,1,0.1,1)]
for i in ratings:
    axis = fig.add_subplot(rows,cols,p)
    sns.distplot(df.Review_CC[df.Ratings==i], ax=axis, label=f"For Rating: {i}", color=colors[i-1])
    axis.set_xlabel(f"Review Character Count")
    axis.legend()
    p += 1
    
plt.show()

In [39]:
# Getting insight of loud words in each rating
cols = 2
ratings = np.sort(df.Ratings.unique())
rows = len(ratings)//2
if len(ratings) % cols != 0:
    rows += 1    
fig = plt.figure(figsize=(15,20))
plt.subplots_adjust(hspace=0.3)
p = 1
for i in ratings:
    word_cloud = WordCloud(height=800, width=1000, background_color="white", max_words=50).generate(' '.join(df.Review[df.Ratings==i]))
    axis = fig.add_subplot(rows,cols,p)
    axis.set_title(f"WordCloud for Rating: {i}\n")
    axis.imshow(word_cloud)
    for spine in axis.spines.values():
        spine.set_edgecolor('r')
    axis.set_xticks([])
    axis.set_yticks([])
    
    plt.tight_layout(pad=5)
    p += 1    
plt.show()

In [40]:
# Checking the count of target column values
plt.figure(figsize=(15,7))
sns.countplot(df['Ratings'])
print(df.Ratings.value_counts())
plt.show()

5    19264
4     7185
1     5855
3     3402
2     2469
Name: Ratings, dtype: int64


In [41]:
# shuffling the entire data set
df = df.sample(frac=1).reset_index(drop=True)
df

,Review_title,Review_text,Ratings,Review,Review_WC,Review_CC
0,Perfect product!,"I just received this product, product is good....",5,perfect product receive product product good p...,56,336
1,Terrific,"Amazing, excellent",5,terrific amaze excellent,3,24
2,Good choice,Gud,4,good choice gud,3,15
3,Brilliant,Nice camera quality,5,brilliant nice camera quality,4,29
4,Amazing Display,This monitor is just awesome. Amazing picture ...,5,amaze display monitor awesome amazing picture ...,46,284
...,...,...,...,...,...,...
38170,Awesome,White colour is most attractive... it's camera...,5,awesome white colour attractive camera quality...,8,65
38171,Super!,It's good product at this price range . Very s...,5,super good product price range sli amand size ...,11,57
38172,Fabulous!,Amazing laptop must buy,5,fabulous amaze laptop must buy,5,30
38173,Great product,Good,5,great product good,3,18


In [42]:
# Select data from every Ratings category
df1 = df[df['Ratings']==1][0:7356]
df2 = df[df['Ratings']==2][0:7356]
df3 = df[df['Ratings']==3][0:7356]
df4 = df[df['Ratings']==4][0:7356]
df5 = df[df['Ratings']==5][0:7356]

# Combining all the dataframes into one and shuffling them again
df = pd.concat([df1,df2,df3,df4,df5], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)
df

,Review_title,Review_text,Ratings,Review,Review_WC,Review_CC
0,Excellent,Nice product,5,excellent nice product,3,22
1,Worth the money,*Display is great.... No such phone can provid...,4,worth money display great phone provide type d...,53,320
2,Could be way better,Bad product....\nNot as expected....\nThis is ...,2,could way well bad product expect data storage...,20,125
3,Wonderful,Nice,5,wonderful nice,2,14
4,Did not meet expectations,Got replacement first one not working 2 nd is ...,1,meet expectation get replacement first one wor...,14,74
...,...,...,...,...,...,...
26262,Does the job,Mobile performance good for normal uses\nCamer...,3,job mobile performance good normal us camera q...,19,126
26263,Delightful,Very good product and light weight.,3,delightful good product light weight,5,36
26264,Connectivity issues,\n So the thing is it kinda sucks at the conn...,3,connectivity issue thing kinda suck connectivi...,42,299
26265,Fair,Smail size,3,fair smail size,3,15


In [43]:
# lets Check the count of target column again to confirm the balance with a visual
plt.figure(figsize=(15,7))
sns.countplot(df['Ratings'])
print(df.Ratings.value_counts())
plt.show()

5    7356
4    7185
1    5855
3    3402
2    2469
Name: Ratings, dtype: int64


In [44]:
# Function to plot most frequent terms in our Review column
def freq_words(x, terms=30):
    all_words = ' '.join([text for text in x])
    all_words = all_words.split()
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()),
                             'count':list(fdist.values())})
    # selecting top 30 most frequent words
    dt = words_df.nlargest(columns='count', n=terms)
    plt.figure(figsize=(15,10))
    ax = sns.barplot(data=dt, x='count', y='word')
    ax.set(ylabel = 'Word')
    plt.show()
    
freq_words(df['Review'])    

In [45]:
# Function to plot least frequent terms in our Review column
def rare_words(x, terms=30):
    all_words = ' '.join([text for text in x])
    all_words = all_words.split()
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()),
                             'count':list(fdist.values())})
    # selecting top 30 least freq rare words
    dt = words_df.nsmallest(columns='count', n=terms)
    plt.figure(figsize=(15,10))
    ax = sns.barplot(data=dt, x='count', y='word')
    ax.set(ylabel = 'Word')
    plt.show()
    
rare_words(df['Review'])    

In [46]:
# Separating features and labels here
x = df['Review']
y = df['Ratings']

In [47]:
# Using the n_gram tfidf vectorizer (Word vectors)
word_vectorizer = TfidfVectorizer(sublinear_tf = True,
                                  strip_accents = 'unicode',
                                  analyzer = 'word',
                                  token_pattern = r'\w{1,}',
                                  stop_words = 'english',
                                  ngram_range = (1,3),
                                  max_features = 100000)
word_vectorizer.fit(x)
train_word_features = word_vectorizer.transform(x)

# Character vectors
char_vectorizer = TfidfVectorizer(sublinear_tf = True,
                                  strip_accents = 'unicode',
                                  analyzer = 'char',
                                  stop_words = 'english',
                                  ngram_range = (2,6),
                                  max_features = 50000)
char_vectorizer.fit(x)
train_char_features = char_vectorizer.transform(x)

# I will now combine both word vectors and character vectors as input for our model
train_features = hstack([train_char_features, train_word_features])

In [48]:
state = 42
x_train, x_test, y_train, y_test = train_test_split(train_features, y, test_size = 0.30, random_state = state)

# Lets check the shapes of traning and test data
print("x_train", x_train.shape)
print("x_test", x_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)

x_train (18386, 150000)
x_test (7881, 150000)
y_train (18386,)
y_test (7881,)


In [49]:
# Defining the Classification Machine Learning Algorithms
rf = RandomForestClassifier()
lr = LogisticRegression(solver='lbfgs')
svc = LinearSVC()
bnb = BernoulliNB()
mnb = MultinomialNB()
sgd = SGDClassifier()
lgb = LGBMClassifier()
xgb = XGBClassifier(verbosity=0)

# Creating a function to train and test the model with evaluation metrics
def BuiltModel(model):
    print('*'*30+model.__class__.__name__+'*'*30)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_train)
    pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, pred)*100
    print(f"ACCURACY SCORE PERCENTAGE:", accuracy)
    # Confusion matrix and Classification report
    print(f"CLASSIFICATION REPORT: \n {classification_report(y_test, pred)}")
    print(f"CONFUSION MATRIX: \n {confusion_matrix(y_test, pred)}\n")
    print("-"*120)
    print("\n")

In [50]:
for model in [lr,svc,bnb,mnb,sgd,rf,xgb,lgb]:
    BuiltModel(model)

******************************LogisticRegression******************************
ACCURACY SCORE PERCENTAGE: 82.2357568836442
CLASSIFICATION REPORT: 
               precision    recall  f1-score   support

           1       0.83      0.92      0.87      1816
           2       0.72      0.53      0.61       738
           3       0.71      0.63      0.67      1028
           4       0.80      0.87      0.83      2124
           5       0.92      0.89      0.90      2175

    accuracy                           0.82      7881
   macro avg       0.80      0.77      0.78      7881
weighted avg       0.82      0.82      0.82      7881

CONFUSION MATRIX: 
 [[1667   64   43   35    7]
 [ 190  394   86   58   10]
 [ 104   64  643  189   28]
 [  34   14  105 1839  132]
 [  17    8   23  189 1938]]

------------------------------------------------------------------------------------------------------------------------


******************************LinearSVC******************************
ACCURACY

In [51]:
# I am checking cross-validation score only for those algorithms which are giving us better accuracies

def cross_val(model):
    print('*'*30+model.__class__.__name__+'*'*30)
    scores = cross_val_score(model,train_features,y, cv = 3).mean()*100
    print("Cross validation score:", scores)
    print("\n")
    
for model in [lr,svc,sgd,rf,lgb,xgb]:
    cross_val(model)    

******************************LogisticRegression******************************
Cross validation score: 81.55860598529398


******************************LinearSVC******************************
Cross validation score: 81.4938997637757


******************************SGDClassifier******************************
Cross validation score: 81.53577102757615


******************************RandomForestClassifier******************************
Cross validation score: 81.15123668808714


******************************LGBMClassifier******************************
Cross validation score: 82.23625482517028


******************************XGBClassifier******************************
Cross validation score: 81.77559048030766




In [52]:
# Lets select the different parameters for tuning our best model (RandomForestClassifier)
grid_params = {'n_estimators':[100,200],
               'criterion':['gini','entropy'],
               'max_depth': [500,800],
               'bootstrap':[True,False]}

# Train the model with given parameters using GridSearchCV
GSCV =  GridSearchCV(rf, grid_params, cv=3, verbose=3)
GSCV.fit(x_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=500, n_estimators=100;, score=0.789 total time= 1.6min
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=500, n_estimators=100;, score=0.791 total time= 1.8min
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=500, n_estimators=100;, score=0.791 total time= 1.9min
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=500, n_estimators=200;, score=0.793 total time= 3.2min
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=500, n_estimators=200;, score=0.793 total time= 3.5min
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=500, n_estimators=200;, score=0.800 total time= 3.8min
[CV 1/3] END bootstrap=True, criterion=gini, max_depth=800, n_estimators=100;, score=0.791 total time= 1.6min
[CV 2/3] END bootstrap=True, criterion=gini, max_depth=800, n_estimators=100;, score=0.786 total time= 1.7min
[CV 3/3] END bootstrap=True, criterion=gini, max_depth=800,

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [500, 800], 'n_estimators': [100, 200]},
             verbose=3)

In [53]:
GSCV.best_params_ # Selecting the best parameters found by GridSearchCV

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 500,
 'n_estimators': 200}

In [54]:
# Final Model with the best chosen parameters list
best_model = RandomForestClassifier(bootstrap=False, criterion="gini", max_depth=800, n_estimators=200)
best_model.fit(x_train,y_train) # fitting data to the best model
pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, pred)*100
# Printing the accuracy score
print("ACCURACY SCORE:", accuracy)
# Printing the classification report
print(f"\nCLASSIFICATION REPORT: \n {classification_report(y_test, pred)}")
# Printing the Confusion matrix
print(f"\nCONFUSION MATRIX: \n {confusion_matrix(y_test, pred)}")

ACCURACY SCORE: 81.7789620606522

CLASSIFICATION REPORT: 
               precision    recall  f1-score   support

           1       0.78      0.95      0.86      1816
           2       0.88      0.45      0.59       738
           3       0.78      0.60      0.68      1028
           4       0.77      0.87      0.82      2124
           5       0.91      0.89      0.90      2175

    accuracy                           0.82      7881
   macro avg       0.82      0.75      0.77      7881
weighted avg       0.82      0.82      0.81      7881


CONFUSION MATRIX: 
 [[1720   19   20   53    4]
 [ 238  331   60   93   16]
 [ 153   16  620  203   36]
 [  59    6   83 1842  134]
 [  34    3   16  190 1932]]


In [55]:
# Creating a normalized confusion matrix here
skplt.metrics.plot_confusion_matrix(y_test, pred, normalize=True)

<AxesSubplot:title={'center':'Normalized Confusion Matrix'}, xlabel='Predicted label', ylabel='True label'>

In [56]:
joblib.dump(best_model, "Rating_Prediction_Saved_Model.pkl")

['Rating_Prediction_Saved_Model.pkl']

In [57]:
Model = joblib.load("Rating_Prediction_Saved_Model.pkl")

In [58]:
# Predicting test data using loaded model
prediction = Model.predict(x_test)
# Analysing Predicted vs Actual results
results = pd.DataFrame()
results['Predicted Review Ratings'] = prediction
results['Actual Review Ratings'] = y
results

,Predicted Review Ratings,Actual Review Ratings
0,2,5
1,5,4
2,5,2
3,5,5
4,1,1
...,...,...
7876,2,4
7877,4,4
7878,5,3
7879,4,2


In [59]:
# Converting the dataframe into CSV format and saving it
results.to_csv('PredictedvsActual_Ratings_Results.csv', index=False)